© Валерий Студенников, курс "Инструменты анализа данных" в СГАУ

# Продвинутые визможности Pandas

pandas — это высокоуровневая Python библиотека для работы с табличными данными.
matplitlib — библиотека Python-библиотека для визуализации данных

Материалы pandas:
* https://pandas.pydata.org/pandas-docs/stable/reshaping.html
* https://pandas.pydata.org/pandas-docs/stable/merging.html
* Книжка *"Python для анализа данных"*, главы про pandas

In [2]:
import numpy as np
import pandas as pd

## Продвинутые возможности pandas

### Иерархические индексы (MultiIndex)

Иерархические индексы позволяют организовать несколько (два и более) уровней индексирования по одной оси. Другими словави, это способ работать с многомерными данными, представив их в форме с меньшей размерностью.

In [3]:
# сгенерируем series с multiindex
data = pd.Series(np.random.randn(10),
    index= [ ['a', 'a', 'a', 'b', 'b', 'b', 'с', 'с', 'd', 'd'],
             [1, 2, 3, 1, 2, 3, 1, 2, 2, 3] ]
)
data

a  1   -1.451615
   2    0.391336
   3    1.827302
b  1   -0.239388
   2   -2.010882
   3   -0.499735
с  1    1.828901
   2    0.708976
d  2   -0.646960
   3    0.740042
dtype: float64

In [6]:
# сгенерируем dataframe с multiindex
data = pd.DataFrame(np.random.randn(10,2),
    index= [ ['a', 'a', 'a', 'b', 'b', 'b', 'с', 'с', 'd', 'd'],
             [1, 2, 3, 1, 2, 3, 1, 2, 2, 3] ]
)
data

0         1
a 1  0.043274  0.015632
  2 -1.253891 -0.667621
  3  0.730579 -0.258850
b 1  0.135608  0.251531
  2 -0.546798  1.123193
  3  1.204709  0.345419
с 1 -0.138410  1.479747
  2  0.559943  0.859334
d 2 -1.462525  1.745590
  3  0.363495  0.116542

In [22]:
data.index

MultiIndex(levels=[['a', 'b', 'd', 'с'], [1, 2, 3]],
           labels=[[0, 0, 0, 1, 1, 1, 3, 3, 2, 2], [0, 1, 2, 0, 1, 2, 0, 1, 1, 2]])

Для иерархически индексированного объекта возможен доступ по так называемому
частичному индексу, что позволяет лаконично записывать выборку подмножества
данных.

In [8]:
# доступ по частичному значению индекса
data.loc['a']

,0,1
1,0.043274,0.015632
2,-1.253891,-0.667621
3,0.730579,-0.258850


In [7]:
# доступ по полному значению индекса
data.loc[('a', 1)]

0    0.043274
1    0.015632
Name: (a, 1), dtype: float64

In [11]:
# Можно даже сразу получить значение поля
data.loc[('a', 1), 1]

0.01563242378492272

In [79]:
# SELECT site, YEAR(regdate) regyear, COUNT(*) usrcnt FROM users u GROUP BY 1,2
df_regusrcnt = pd.read_csv('/tmp/regreg.csv', sep = '\t', index_col = ['site', 'regyear'])
df_regusrcnt.head(10)

usrcnt
site             regyear        
100mb.ru         2013       1218
                 2014          2
                 2015          2
agava.ru         2016      40622
globatel.ru      2016         67
hostingjoomla.ru 2012        331
                 2013          9
                 2014          1
logol.ru         2014      16589
mne.ru           2017       6904

In [80]:
df_regusrcnt.index

MultiIndex(levels=[['100mb.ru', 'agava.ru', 'globatel.ru', 'hostingjoomla.ru', 'logol.ru', 'mne.ru', 'regru', 'test-regru'], [2006, 2007, 2008, 2009, 2010, 2011, 2012, 2013, 2014, 2015, 2016, 2017, 2018]],
           labels=[[0, 0, 0, 1, 2, 3, 3, 3, 4, 5, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 7, 7], [7, 8, 9, 10, 10, 6, 7, 8, 8, 11, 0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 10, 11]],
           names=['site', 'regyear'])

In [81]:
df_regusrcnt.swaplevel( i = 0, j = 1, axis = 0 ).sort_index()

usrcnt
regyear site                    
2006    regru                  8
2007    regru               2663
2008    regru              13834
2009    regru              22956
2010    regru              67581
2011    regru              94837
2012    hostingjoomla.ru     331
        regru             108138
2013    100mb.ru            1218
        hostingjoomla.ru       9
        regru             123686
2014    100mb.ru               2
        hostingjoomla.ru       1
        logol.ru           16589
        regru             147402
2015    100mb.ru               2
        regru             214477
2016    agava.ru           40622
        globatel.ru           67
        regru             281183
        test-regru            39
2017    mne.ru              6904
        regru             255002
        test-regru             1
2018    regru             289743

### df.pivot, df.pivot_table + MultiIndex

![.pivot](https://pandas.pydata.org/pandas-docs/stable/_images/reshaping_pivot.png)

In [45]:
# Сбросим индексы
df_regusrcnt.reset_index().head(7)

,site,regyear,usrcnt
0,100mb.ru,2013,1218
1,100mb.ru,2014,2
2,100mb.ru,2015,2
3,agava.ru,2016,40622
4,globatel.ru,2016,67
5,hostingjoomla.ru,2012,331
6,hostingjoomla.ru,2013,9


In [49]:
df_pivot = df_regusrcnt.reset_index().pivot( index = 'site', columns = 'regyear', values = 'usrcnt' )
del df_pivot[0]
df_pivot.fillna(0).astype(int)

regyear,2006,2007,2008,2009,2010,2011,2012,2013,2014,2015,2016,2017,2018
site,,,,,,,,,,,,,
100mb.ru,0,0,0,0,0,0,0,1218,2,2,0,0,0
agava.ru,0,0,0,0,0,0,0,0,0,0,40622,0,0
globatel.ru,0,0,0,0,0,0,0,0,0,0,67,0,0
hostingjoomla.ru,0,0,0,0,0,0,331,9,1,0,0,0,0
logol.ru,0,0,0,0,0,0,0,0,16589,0,0,0,0
mne.ru,0,0,0,0,0,0,0,0,0,0,0,6904,0
regru,8,2663,13834,22956,67581,94837,108138,123686,147402,214477,281183,255002,289513
test-regru,0,0,0,0,0,0,0,0,0,0,39,1,0


In [50]:
# SELECT site, YEAR(regdate) regyear, country_code, COUNT(*) usrcnt FROM users u WHERE country_code > "" GROUP BY 1,2;
df_regusrcnt2 = pd.read_csv('/tmp/regreg2.csv', sep = '\t')
df_regusrcnt2.head(10)

,site,regyear,country_code,usrcnt
0,100mb.ru,2013,RU,1218
1,100mb.ru,2014,RU,2
2,100mb.ru,2015,RU,2
3,agava.ru,2016,RU,40166
4,globatel.ru,2016,RU,67
5,hostingjoomla.ru,2012,RU,331
6,hostingjoomla.ru,2013,RU,9
7,hostingjoomla.ru,2014,RU,1
8,logol.ru,2014,RU,16395
9,mne.ru,2017,RU,6904


In [55]:
df_regusrcnt2.pivot_table(
    index = ['site', 'country_code'], columns = 'regyear', values = 'usrcnt'
).fillna(0).astype(int)

regyear                        0     2006  2007   2008   2009   2010   2011  \
site             country_code                                                 
100mb.ru         RU               0     0     0      0      0      0      0   
agava.ru         RU               0     0     0      0      0      0      0   
globatel.ru      RU               0     0     0      0      0      0      0   
hostingjoomla.ru RU               0     0     0      0      0      0      0   
logol.ru         RU               0     0     0      0      0      0      0   
mne.ru           RU               0     0     0      0      0      0      0   
regru            BG               0     0     0      0      0      0      0   
                 CA               0     0     0      0      0      0  94837   
                 MD               0     0     0      0  22955      0      0   
                 RU               1     8  2663  13834      0  67581      0   
test-regru       RU               0     0     0      0      0      0      0   

regyear                          2012    2013    2014    2015    2016    2017  \
site             country_code                                                   
100mb.ru         RU                 0    1218       2       2       0       0   
agava.ru         RU                 0       0       0       0   40166       0   
globatel.ru      RU                 0       0       0       0      67       0   
hostingjoomla.ru RU               331       9       1       0       0       0   
logol.ru         RU                 0       0   16395       0       0       0   
mne.ru           RU                 0       0       0       0       0    6904   
regru            BG                 0       0       0       0       0  255002   
                 CA                 0       0       0       0       0       0   
                 MD                 0       0       0       0       0       0   
                 RU            108137  123686  147402  214477  281183       0   
test-regru       RU                 0       0       0       0      39       1   

regyear                          2018  
site             country_code          
100mb.ru         RU                 0  
agava.ru         RU                 0  
globatel.ru      RU                 0  
hostingjoomla.ru RU                 0  
logol.ru         RU                 0  
mne.ru           RU                 0  
regru            BG                 0  
                 CA                 0  
                 MD                 0  
                 RU            249966  
test-regru       RU                 0

## .merge

Слияние таблиц по индексуц

In [13]:
df1 = pd.DataFrame( -np.random.random((5,2)), columns = ['A', 'B'] )
df1

,A,B
0,-0.841165,-0.517022
1,-0.971720,-0.350440
2,-0.525301,-0.628533
3,-0.673339,-0.355046
4,-0.239472,-0.156176


In [14]:
df2 = pd.DataFrame( np.random.exponential(100, (4,2)), columns = ['C', 'D'] )
df2

,C,D
0,81.478181,6.905577
1,222.392058,289.719057
2,49.226365,8.585860
3,24.804069,184.394131


In [15]:
# по умолчанию у нас inner join
df1.merge( df2, left_index = True, right_index = True )

,A,B,C,D
0,-0.841165,-0.517022,81.478181,6.905577
1,-0.971720,-0.350440,222.392058,289.719057
2,-0.525301,-0.628533,49.226365,8.585860
3,-0.673339,-0.355046,24.804069,184.394131


In [16]:
# сделаем left join
df1.merge( df2, left_index = True, right_index = True, how = 'left' )

,A,B,C,D
0,-0.841165,-0.517022,81.478181,6.905577
1,-0.971720,-0.350440,222.392058,289.719057
2,-0.525301,-0.628533,49.226365,8.585860
3,-0.673339,-0.355046,24.804069,184.394131
4,-0.239472,-0.156176,NaN,NaN
